<a href="https://colab.research.google.com/github/CresGallego/neural_network/blob/main/neural_network_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dependencies and needed packages
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random as rand
import time

# desired accuracy of the model (you can change this, the higher the longer for model to train)
accuracy_target = 0.96

# load data from Mnist
data = tf.keras.datasets.fashion_mnist

# extract data to training and test data sets using load_data() method inherent from data object
# data.load() method returns 2 tuples (first tuple has 1 array for the training_images and the other
# array is for the training_labels, the 2nd tuple is for the test_images and test_labels array)
# it is the method that splits this into 60k images for first tuple and 10k images for 2nd tuple
(training_images, training_labels),(test_images, test_labels) = data.load_data()

# normalize the data, we use 255.0 as it is the range of grayscale of the pixel,
# so the result is normalized into floats (range 0 to 1)instead of 0 to 255 pixel value range
training_images = training_images / 255.0
test_images = test_images / 255.0


# create a callback to stop the training of the model when accuracy reached 93% (to avoid over fitting)
# the callback is defined here and will be used as a parameter later in training the model
# the purpose is to stop at certain point in training (epochs) when it achieved the desired accuracy and so need not continue all the epochs required
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epochs,logs={}):
    if(logs.get('accuracy')>accuracy_target):
      self.model.stop_training = True
      print(f"\nHave reached the {accuracy_target*100:}% accuracy, so halting the the training!")

callback = myCallback()


# DESIGN THE NEURAL NETWORK MODEL, inherits from keras APIs
# Sequential is most common model to use, it executes the algorithms one after the other
# Flatten to convert the 2D array of the image pixels which in this case a small 28x28 pixel picturei into an array
# Input layer has 128 neurons (arbitrary but good number, not low not high), activation function is Rectified Linear Unit,simply to drop negatives to zero
# Output layer has 10 neurons to correspond to the number of categories , activation function is SoftMax forcing output into probability values
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# COMPILE THE MODEL, with 3 parameters
# optimizer is Adaptive Moment Estimation 'adam' is good for this case as it is faster and efficient for training
# loss function is sparse_categorical_crossentropy, appropriate as it is memory-efficient for large class datasets
# we need to see how accuracy goes for each epoch
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# TRAIN THE MODEL, with callback, we feed the training_images as x parameter and training_labels as y parameter,
# arbitrarily we repeat the data upto 50 times, unless stop by callback
# we capture the time before and after just to see how long is the training takes
start_time = time.time()
model.fit(training_images, training_labels, epochs=50,callbacks=[callback] )
end_time = time.time()
print("\nTraining took {} seconds to reach the {}% accuracy desired.".format(round(end_time-start_time), round(accuracy_target*100)))

# EVALUATE THE MODEL
loss, accuracy = model.evaluate(test_images, test_labels)
print("Evaluation Result")
print(f"Accuracy : {accuracy*100:.2f}%")
print(f"Loss     : {loss*100:.2f}%")



In [ ]:
# RUN PREDICTIONS
# to randomly use any of the 10,000 test images for prediction, make avariable to store the random imge number
test_image_number = rand.randint(0, 10000)
print(f"Test image number : {test_image_number}")

# variable to store the array of the 10 category names
category_name = ["T-Shirt", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag","Ankle Boot"]
print(f"Category Names : {category_name}")

# variable to store the array of 10 probabilities of randomly selected image being predicted by calling the predict() method of the model
classifications = model.predict(test_images)

# using numpy's argmax() method to get the index number of the highest probability in classification array
predicted_label = np.argmax(classifications[test_image_number])
print(f"Predicted label index number :{predicted_label}")

# extract predicted label name using the predicted_label as the index to locate the position in the array of category name
predicted_label_name = category_name[predicted_label]
print(f"Predicted label name : {predicted_label_name}")

# to extract the actual label (number 0 to 9) of the image (of test dataset) being predicted
test_label = test_labels[test_image_number]
print(f"Actual label : {test_label}")

# to extract the actual label name of the image
actual_label_name = category_name[test_label]
print(f"Actual label name : {actual_label_name}")

# to get the value of the probability scorein classification array using test_image_number as outer index and predicted_
probability = round(classifications[test_image_number][predicted_label]*100,1)
print(f"Probability : {probability}%")

# define a function to tell if the prediction is correct or not, probability
def is_predict_correct(predicted_label, test_label):
  if predicted_label == test_label:
    answer = "Correct"
  else:
    answer ="Wrong"

  return answer

answer = is_predict_correct(predicted_label, test_label) #unpack the return values and save as variables

print("\nThe probabilities of classifications for test image number '{}' is the following:\n{}".format(test_image_number,classifications[test_image_number]))
print("\nTest image number '{}' is predicted label to be '{}' and the actual label is '{}'.".format(test_image_number, predicted_label, test_label))
print("Test image number '{}' predicted name is '{}' and the actual name is '{}'!".format(test_image_number, predicted_label_name, actual_label_name))
print("Prediction is '{}' and with '{}%' probability".format(answer, probability))

# show the predicted image
plt.figure(figsize=(2,2))
plt.imshow(test_images[test_image_number])
plt.xticks([])
plt.yticks([])
plt.title("Image Num : {}". format(test_image_number))
plt.xlabel("Predicted : {}\nPrediction : {}\nProbability : {}%".format(predicted_label_name, answer, probability))
plt.ylabel("Actual : {}".format(actual_label_name))
plt.show()